Initialize CIFAR10 data

Please upload the files in `func_simCIFAR10` to the `/home` directory

In [ ]:
%pip install avalanche-lib
%cd /home   
%run download_data.py
%run generate_txt.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 KB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.4/532.4 KB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.7/455.7 KB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.4 MB/s eta 0:00:00
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=1625f5967

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/cifar10/cifar-10-python.tar.gz to /root/.avalanche/data/cifar10
Files already downloaded and verified


In [ ]:
import torch 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
"""
calculate the functional similarity between task 1 (distinguish digits airplane and automobile) and others
"""
import torch
import os
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision.models.resnet import resnet50


from util import trainES,get_Cifar10

# ------------------------------------ step 0/5 : initialise hyper-parameters ------------------------------------
basic_task = 0  # count from 0
experience = 5
train_bs = 128
test_bs = 128
lr_init = 0.001
max_epoch = 500
run_times = 10
patience = 20

# 是否使用GPU
no_cuda = False
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
kwargs = {"num_workers":1, "pin_memory":True} if use_cuda else {}

fun_score = np.zeros((run_times, 4))
for run in range(run_times):
    print("run time: {}".format(run+1))
# ------------------------------------ step 1/5 : load data------------------------------------
    train_stream, test_stream = get_Cifar10()
# ------------------------------------ step 2/5 : define network-------------------------------
    model = resnet50()
    model.fc = nn.Linear(model.fc.in_features, 2)
    # ------------------------------------ step 3/5 : define loss function and optimization ------------------------
    criterion = nn.CrossEntropyLoss()  
    optimizer = optim.SGD(model.parameters(), lr=lr_init)  
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1) 
    # ------------------------------------ step 4/5 : training --------------------------------------------------
    # training basic task
    train_data = train_stream[basic_task]
    test_data = test_stream[basic_task]
    model,_ , _ ,avg_valid_losses , _ = trainES(train_data,test_data,model,criterion,optimizer,max_epoch,device,patience = 20)
    basic_loss = avg_valid_losses[-1]
    print("basic loss:{:.4}".format(basic_loss))
    # save task 1
    PATH = "./"
    trained_model_path = os.path.join(PATH, 'model4sim.pth')
    torch.save(model.state_dict(), trained_model_path)

    # pop the src data from train_stream and test_stream
    train_stream.pop(basic_task)
    test_stream.pop(basic_task)
    # for other tasks , load task 1
    for j, probe_data in enumerate(train_stream):
        # load old task's model
        trained_model = resnet50()
        trained_model.fc = nn.Linear(trained_model.fc.in_features, 2) # final output dim = 2
        trained_model.load_state_dict(torch.load(trained_model_path))
        criterion = nn.CrossEntropyLoss()  
        optimizer = optim.SGD(trained_model.parameters(), lr=lr_init, momentum=0.9, dampening=0.1)  
        train_data = probe_data
        # load other tasks
        for k, batch in enumerate(train_data):  
            x_train, y_train = batch[0].to(device), batch[1].to(device)
            trained_model.to(device)
            y_pred = trained_model(x_train)
            loss = criterion(y_pred, y_train)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # calculate the functional similarity (fun_score) at last steps
            if k == 0:
                print(loss.item())
                fun_score[run, j] = (1 - (loss.item() / basic_loss.item()))
                break
    print(fun_score[run, :])   
fun_score_mean = np.mean(fun_score, axis=0)
fun_score_std = np.std(fun_score, axis=0)
print(fun_score_mean)
print(fun_score_std)


run time: 1
[  0/500]  train_loss: 0.76750 valid_loss: 2.38344 train_acc: 0.52828 valid_acc: 0.56240
Validation loss decreased (inf --> 2.383442).  Saving model ...
[  1/500]  train_loss: 0.70688 valid_loss: 1.29705 train_acc: 0.55518 valid_acc: 0.57075
Validation loss decreased (2.383442 --> 1.297051).  Saving model ...
[  2/500]  train_loss: 0.69524 valid_loss: 0.86443 train_acc: 0.56899 valid_acc: 0.57783
Validation loss decreased (1.297051 --> 0.864428).  Saving model ...
[  3/500]  train_loss: 0.68179 valid_loss: 0.74579 train_acc: 0.58052 valid_acc: 0.58733
Validation loss decreased (0.864428 --> 0.745787).  Saving model ...
[  4/500]  train_loss: 0.66269 valid_loss: 0.66302 train_acc: 0.58958 valid_acc: 0.59309
Validation loss decreased (0.745787 --> 0.663016).  Saving model ...
[  5/500]  train_loss: 0.64386 valid_loss: 0.94850 train_acc: 0.59851 valid_acc: 0.59302
EarlyStopping counter: 1 out of 20
[  6/500]  train_loss: 0.63398 valid_loss: 0.62173 train_acc: 0.60584 valid_acc

the mean and std of functional similarity, task 1 and 5 have the cloest inter-task relatedness

In [ ]:
fun_score_mean 

array([-1.17412479, -1.51228824, -1.88969593, -0.40121193])

In [ ]:
fun_score_std

array([0.16085158, 0.41443986, 0.22952561, 0.14195573])

In [ ]:
"""
learn tasks in 2 phases
"""
import torch
import os
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision.models.resnet import resnet50

from util import trainES,get_Cifar10,test
from metrics import compute_acc_fgt

# ------------------------------------ step 0/5 : initalzation------------------------------------
basic_task = 0
experience = 5
train_bs = 128
test_bs = 128
lr_init = 0.001
max_epoch = 500
run_times = 6
patience = 20

accuracy_list1 = [] # multiple run
accuracy_list2 = []
accuracy_list3 = []
accuracy_list4 = []

no_cuda = False
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
kwargs = {"num_workers":1, "pin_memory":True} if use_cuda else {}

fun_score = np.zeros((run_times, 4))
for run in range(run_times):
    print("run time: {}".format(run+1))
# ------------------------------------ step 1/5 : load data------------------------------------
    train_stream, test_stream = get_Cifar10()
# ------------------------------------ step 2/5 : define network------------------------------------
    model = resnet50()
    model.fc = nn.Linear(model.fc.in_features, 2)
    # ------------------------------------ step 3/5 : define loss function and optim ------------------------------------=
    criterion = nn.CrossEntropyLoss()  
    optimizer = optim.SGD(model.parameters(), lr=lr_init)  
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)  
    # ------------------------------------ step 4/5 : training --------------------------------------------------
    # training basic task
    basic_task_data = train_stream[basic_task]
    basic_task_test_data = test_stream[basic_task]
    model, _, _, _, _ = trainES(basic_task_data, basic_task_test_data, model, criterion, optimizer, max_epoch, device,
                                patience)
    # setting stage 1 matrix
    acc_array1 = np.zeros((4, 2))
    # testing basic task
    _, acc_array1[:, 0] = test(test_stream[basic_task], model, criterion, device)
    # pop basic task
    train_stream.pop(basic_task)
    test_stream.pop(basic_task)
    # test other tasks except basic task
    for i, probe_data in enumerate(test_stream):
        with torch.no_grad():
            _, acc_array1[i, 1] = test(probe_data, model, criterion, device)
    # save task 1
    PATH = "./"
    trained_model_path = os.path.join(PATH, 'modle4acc_fgt.pth')
    torch.save(model.state_dict(), trained_model_path)
    # setting stage 2 matrix
    acc_array2 = np.zeros((4, 2))
    for j, (train_data, test_data) in enumerate(zip(train_stream, test_stream)):
        print("task {} starting...".format(j))
        # load old task's model
        # trained_model = MLP(out_dim=2, in_channel=in_channel)
        trained_model = resnet50()
        trained_model.fc = nn.Linear(trained_model.fc.in_features, 2) # final output dim = 2
        trained_model.load_state_dict(torch.load(trained_model_path))
        criterion = nn.CrossEntropyLoss()  
        optimizer = optim.SGD(trained_model.parameters(), lr=lr_init, momentum=0.9, dampening=0.1)  
        # training other tasks
        trained_model, _, _, _, _ = trainES(train_data, test_data, trained_model, criterion, optimizer, max_epoch,
                                            device, patience)
        # test model on basic task and task j
        with torch.no_grad():
            _, acc_array2[j, 0] = test(basic_task_test_data, trained_model, criterion, device)
            _, acc_array2[j, 1] = test(test_stream[j], trained_model, criterion, device)
    # computing avg_acc and CF
    accuracy_list1.append([acc_array1[0, :], acc_array2[0, :]])
    accuracy_list2.append([acc_array1[1, :], acc_array2[1, :]])
    accuracy_list3.append([acc_array1[2, :], acc_array2[2, :]])
    accuracy_list4.append([acc_array1[3, :], acc_array2[3, :]])

accuracy_array1 = np.array(accuracy_list1)
accuracy_array2 = np.array(accuracy_list2)
accuracy_array3 = np.array(accuracy_list3)
accuracy_array4 = np.array(accuracy_list4)

avg_end_acc, avg_end_fgt, avg_acc = compute_acc_fgt(accuracy_array1)
print('----------- Avg_End_Acc {} Avg_End_Fgt {} Avg_Acc {}-----------'.format(avg_end_acc, avg_end_fgt, avg_acc))

avg_end_acc, avg_end_fgt, avg_acc = compute_acc_fgt(accuracy_array2)
print('----------- Avg_End_Acc {} Avg_End_Fgt {} Avg_Acc {}-----------'.format(avg_end_acc, avg_end_fgt, avg_acc))
avg_end_acc, avg_end_fgt, avg_acc = compute_acc_fgt(accuracy_array3)
print('----------- Avg_End_Acc {} Avg_End_Fgt {} Avg_Acc {}-----------'.format(avg_end_acc, avg_end_fgt, avg_acc))
avg_end_acc, avg_end_fgt, avg_acc = compute_acc_fgt(accuracy_array4)
print('----------- Avg_End_Acc {} Avg_End_Fgt {} Avg_Acc {}-----------'.format(avg_end_acc, avg_end_fgt, avg_acc))


run time: 1
[  0/500]  train_loss: 0.75236 valid_loss: 2.94237 train_acc: 0.53837 valid_acc: 0.51602
Validation loss decreased (inf --> 2.942367).  Saving model ...
[  1/500]  train_loss: 0.68386 valid_loss: 0.74442 train_acc: 0.57031 valid_acc: 0.57212
Validation loss decreased (2.942367 --> 0.744420).  Saving model ...
[  2/500]  train_loss: 0.65800 valid_loss: 0.68289 train_acc: 0.58841 valid_acc: 0.59232
Validation loss decreased (0.744420 --> 0.682887).  Saving model ...
[  3/500]  train_loss: 0.62815 valid_loss: 0.62714 train_acc: 0.60302 valid_acc: 0.61074
Validation loss decreased (0.682887 --> 0.627143).  Saving model ...
[  4/500]  train_loss: 0.61936 valid_loss: 0.61320 train_acc: 0.61464 valid_acc: 0.62445
Validation loss decreased (0.627143 --> 0.613197).  Saving model ...
[  5/500]  train_loss: 0.60991 valid_loss: 0.60786 train_acc: 0.62386 valid_acc: 0.63563
Validation loss decreased (0.613197 --> 0.607856).  Saving model ...
[  6/500]  train_loss: 0.60120 valid_loss: 0.